In [1]:
# %% Import libs
import numpy as np
import types
import sys,time
import h5py
import itertools
import imageio
import copy
import re

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.pyplot as plt

from scipy.linalg import expm
from math import factorial
from copy import deepcopy
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import sph_harm
from scipy.optimize import curve_fit
from matplotlib.colors import Normalize


# %% Defining functions
# σ matrices

%matplotlib auto

I = np.array([[1,0],[0,1]])
sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,-1j], [1j,0]])
sigma_z = np.array([[1,0],[0,-1]])
n = np.array([[0,0],[0,1]])

def kronecker_delta(x,y):
    if abs(x-y)<1e-5: return 1
    else: return 0


def snfit(x,A,C,w):
    return C+A/(x/w)**(1/2)

def mymatrixproduct(A,B):
    length = len(A)
    resmat = np.zeros((length,length),dtype = complex)
    for i in range(length):
        for j in range(length):
            resmat[i,j] = A[i]@B[:,j]
            print("\r", end="")
            print(f"{i},{j}", end="")
            sys.stdout.flush()
    return resmat

def myoperation(A,psi):
    length = len(psi)
    res_psi = np.zeros(length.shape,dtype = complex)
    for i in range(length):
        res_psi[i] = A[i]@psi
    return res_psi

# Defining Physical constants
hbar = 6.62607015*(10**(-34))/2/np.pi   # Yet I'd like to set hbar = 1 in the calculation follows
# Defining Atomic Constants
Gamma = 2*np.pi*6*10**6     # 6MHz linewidth of the ^{87}Rb upper state 5P3/2
kappa = 2*np.pi*0.1*10**6   # 0.1MHz linewidth of the cavity
kappa0 = kappa
Omega_a = 2*np.pi*6.834*10**9 # 6.83Ghz energy gap for up and down state
k0w0 = (np.pi*2/(780*10**(-9)))*9.6*10**(-6)

def husimi_q(state, theta, phi):
    q_func = np.zeros((len(theta), len(phi)), dtype=np.complex128)
    for j, t in enumerate(theta):
        for k, p in enumerate(phi):
            alpha = np.cos(t / 2) * np.exp(1j * p / 2)
            beta = np.sin(t / 2) * np.exp(-1j * p / 2)
            coherent_state = np.array([alpha ** m * beta ** (atom_number - m) * np.sqrt(factorial(atom_number) / (factorial(m) * factorial(atom_number - m))) for m in range(atom_number + 1)]).reshape(-1, 1)
            q_func[k, j] = np.abs(np.conj(coherent_state).T @ state) ** 2
    return np.abs(q_func)

class photons():
    def __init__(self,typ,order,delta,t):
        self.order = order
        delta[np.abs(delta)<1] = 0
        self.delta = delta*1.0
        self.t = t
        self.typ = typ

class sequence():
    def __init__(self,photons):
        self.photons = photons



class cavity_atom_light_system():   # Note that here most of the parameters are alterable, except for the atomic parameters.
    def __init__(self, atom_number, cooperativity, Delta, Omega):
        self.atom_number = atom_number              
        self.cooperativity = cooperativity              
        self.Delta = Delta                              # Delta should be much smaller than 6.8GHz/2, otherwise self.t() no longer holds
        self.omega_s = self.cooperativity*kappa*Gamma / (4*self.Delta)     # omega_s = g^2/Δ = ηκΓ/(4Δ)
        self.Omega = Omega                              # Note that we set Omega as a fixed parameter here.
        self.state_index = np.arange(self.atom_number+1)[::-1]    # The first one being the Dicke state with spin_z==+N/2
        # Calculating S_xyz, be ware of the order of Dicke states
        self.S_z = np.diag(np.array([self.atom_number/2 - iter for iter in np.arange(atom_number+1)]))
        S_plus = np.zeros((self.atom_number+1,self.atom_number+1),dtype = complex)
        for i in range(self.atom_number):
            S_plus[i,i+1] = np.sqrt(atom_number/2 * (atom_number/2 +1) - (-atom_number/2+i) * (-atom_number/2+i+1 ) )
        S_minus = np.zeros((self.atom_number+1,self.atom_number+1),dtype = complex)
        for i in range(self.atom_number):
            S_minus[i+1,i] = np.sqrt(atom_number/2 * (atom_number/2 +1) - (-atom_number/2+i) * (-atom_number/2+i+1 ) )
        self.S_plus = S_plus
        self.S_minus = S_minus
        self.S_x = (S_plus + S_minus) / 2
        self.S_y = (S_plus - S_minus) / (2j)

        self.psi0 = self.initialize_psi0()
        self.psi = deepcopy(self.psi0)  # Remember to reset psi after each evolution turn
        self.rho = self.psi@np.transpose(np.conj(self.psi))

    def initialize_psi0(self):      # Initialize a CSS orienting at the north pole, that is, [1,0,0,0,...,0] in the Dicke representation
        psi0 = np.zeros(self.atom_number+1)
        psi0[0] = 1
        return psi0.reshape(-1,1)     # State vectors are column vectors, of course
    
    def H_AC(self, delta, order): # "delta" is the depth of the AC stark shift, and "order" is the Dicke state order with which the AC light resonates
        H_prime_exp = delta*(1 - 1j*Gamma / (2*self.Delta) ) * np.diag([n*np.abs(self.t(order*self.omega_s, n))**2 for n in range(self.atom_number+1)[::-1]]) / (order * np.abs(self.t(order * self.omega_s, order))**2 )   # Check that the first one being +N/2
        return H_prime_exp  # order = 1 recovers the Eq.6 in the main text of Creation of Greenberger-Horne-Zeilinger states with thousands of atoms by entanglement amplification


    def show_H_AC_photons(self,sequence):
        photons = sequence.photons
        sequence_length = len(sequence.photons)
        H_ACs = np.array([[0 for i in range(self.atom_number+1)]],dtype = float)
        for i in range(sequence_length):
            H_AC = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
            for j in range(len(photons[i].delta)):
                H_AC += self.H_AC(photons[i].delta[j], photons[i].order[j])
            H_ACs = np.concatenate((H_ACs,np.array([np.diagonal(H_AC)])))
        H_ACs = np.delete(H_ACs,0,0)
        return H_ACs
    
        

    def evolve_photons(self,sequence):
        photons = sequence.photons
        sequence_length = len(sequence.photons)
        for i in range(sequence_length):
            H_AC = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
            for j in range(len(photons[i].delta)):
                H_AC += self.H_AC(photons[i].delta[j], photons[i].order[j])
            if photons[i].typ == "x":
                self.psi = expm(-1j*(self.Omega*self.S_x + H_AC)*photons[i].t)@ self.psi
            elif photons[i].typ == "z":
                self.psi = expm(-1j*(self.Omega*self.S_z + H_AC)*photons[i].t)@ self.psi
            elif photons[i].typ == "y":
                self.psi = expm(-1j*(self.Omega*self.S_y + H_AC)*photons[i].t)@ self.psi
            else:
                return "Error, wrong input for S type."
        self.rho = self.psi@np.transpose(np.conj(self.psi))
    
    def evolve_photons_t(self,sequence,t_view):
        self.system_reset()
        photons = sequence.photons
        sequence_length = len(photons)
        t0 = 0
        marker = 0
        for i in range(sequence_length):
            if t_view>t0:
                t0 += photons[i].t
                marker += 1
            else:
                break
        if marker != 0:
            for i in range(marker-1):
                H_AC = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
                for j in range(len(photons[i].delta)):
                    H_AC += self.H_AC(photons[i].delta[j], photons[i].order[j])
                if photons[i].typ == "x":
                    self.psi = expm(-1j*(self.Omega*self.S_x + H_AC)*photons[i].t)@ self.psi
                elif photons[i].typ == "z":
                    self.psi = expm(-1j*(self.Omega*self.S_z + H_AC)*photons[i].t)@ self.psi
                elif photons[i].typ == "y":
                    self.psi = expm(-1j*(self.Omega*self.S_y + H_AC)*photons[i].t)@ self.psi
                else:
                    return "Error, wrong input for S type."
            H_AC = self.H_AC(photons[marker-1].delta[0], photons[i].order[0])*0
            t_pre = 0
            for j in range(marker-1):
                t_pre += photons[j].t
            for j in range(len(photons[marker-1].delta)):
                H_AC += self.H_AC(photons[marker-1].delta[j], photons[marker-1].order[j])
            if photons[marker-1].typ == "x":
                self.psi = expm(-1j*(self.Omega*self.S_x + H_AC)*(t_view-t_pre))@ self.psi
            elif photons[marker-1].typ == "z":
                self.psi = expm(-1j*(self.Omega*self.S_z + H_AC)*(t_view-t_pre))@ self.psi
        theta = np.linspace(0, np.pi, 100)
        phi = np.linspace(0, np.pi*4/2, 100)
        self.rho = self.psi@np.transpose(np.conj(self.psi))
        return None
    

    def PoisonMat(self,k1,k,order):
        Np1 = len(self.psi)
        poisonmat = np.zeros((Np1,Np1),dtype = complex)
        for n in range(Np1):
            kn = k1*np.abs(self.t(order*self.omega_s, n)/self.t(order*self.omega_s, order))**2
            if np.abs(kn-k)>100:
                poisonmat[n,n] = 0
            elif k>50:
                poisonmat[n,n] = np.sqrt((kn/k)**k/np.sqrt(2*np.pi*k)*np.exp(k-kn))
            else:
                poisonmat[n,n] = np.sqrt(kn**k/np.math.factorial(k)*np.exp(-kn))
        return poisonmat

    def evolve_photons_IL(self,sequence):
        photons = sequence.photons
        sequence_length = len(sequence.photons)
        for i in range(sequence_length):
            H_AC = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
            knum = len(photons[i].delta)
            kmaxs = np.zeros(knum,dtype = int)
            for j in range(knum):
                kmaxs[j] = 4+1.1*np.abs(int(photons[i].delta[j]/self.omega_s))
            print(kmaxs)
            for j in range(len(photons[i].delta)):
                H_AC += self.H_AC(photons[i].delta[j], photons[i].order[j])
            if photons[i].typ == "x":
                U = expm(-1j*(self.Omega*self.S_x + H_AC)*photons[i].t)
            elif photons[i].typ == "z":
                U = expm(-1j*(self.Omega*self.S_z + H_AC)*photons[i].t)
            elif photons[i].typ == "y":
                U = expm(-1j*(self.Omega*self.S_y + H_AC)*photons[i].t)
            else:
                return "Error, wrong input for S type."
            self.rho = U@self.rho@np.conjugate(np.transpose(U))
            labelmax = 1
            for j in range(knum):
                labelmax *= kmaxs[j]
            ks = np.zeros(knum,dtype = int)
            rhocopy = self.rho*1.0

            rho0 = np.zeros([self.atom_number+1,self.atom_number+1],dtype = complex)
            for l in range(labelmax):
                poisonmat = np.ones([self.atom_number+1,self.atom_number+1],dtype = complex)
                for j in range(knum):
                    modnum = 1
                    for k in range(j+1):
                        modnum *= kmaxs[k]
                    ks[j] = int(np.mod(l/modnum*kmaxs[j],kmaxs[j]))
                    poisonmat *= self.PoisonMat(np.abs(int(photons[i].delta[j]/self.omega_s)),ks[j],photons[i].order[j])
                rho0 += poisonmat@rhocopy@poisonmat
            
            self.rho = rho0

    def Lindblad(self,nc,sequence,plot = False):
        rho = self.rho
        rho_supper = np.zeros((self.atom_number+1)**2*nc**2,dtype = complex)
        lenrs = len(rho_supper)
        for ind1 in range(lenrs):
            i1 = int(ind1/nc**2/(self.atom_number+1))
            j1 = int(np.mod(ind1/nc**2,(self.atom_number+1)))
            n1 = int(np.mod(ind1/nc,nc))
            m1 = int(np.mod(ind1,nc))
            if n1 == 0 and m1 == 0:
                rho_supper[ind1] = rho[i1,j1]
        photons = sequence.photons
        sequence_length = len(sequence.photons)
        num_phoc = np.array([],dtype = float)
        image_list = []
        for i in range(sequence_length):
            rotation = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
            rho0_supper = rho_supper*1.0
            
            no = photons[i].order[0]
            n0 = photons[i].delta[0]/self.omega_s

            if photons[i].typ == "x":
                rotation += self.Omega*self.S_x
            elif photons[i].typ == "z":
                rotation += self.Omega*self.S_z
            elif photons[i].typ == "y":
                rotation += self.Omega*self.S_y
            H_supper = np.zeros([lenrs,lenrs],dtype = complex)
            for ind1 in range(lenrs):
                i1 = int(ind1/nc**2/(self.atom_number+1))
                j1 = int(np.mod(ind1/nc**2,(self.atom_number+1)))
                n1 = int(np.mod(ind1/nc,nc))
                m1 = int(np.mod(ind1,nc))
                for ind2 in range(lenrs):
                    i2 = int(ind2/nc**2/(self.atom_number+1))
                    j2 = int(np.mod(ind2/nc**2,(self.atom_number+1)))
                    n2 = int(np.mod(ind2/nc,nc))
                    m2 = int(np.mod(ind2,nc))
                    if i1 == i2 and j1 == j2 and n1 == n2 and m1 == m2:
                        H_supper[ind1,ind2] = -kappa*(n1+m1)/2-1j*(self.omega_s*((i1-no)*n1-(j2-no)*m2))*(1-1j*Gamma/(2*self.Delta))
                    elif i1 == i2 and j1 == j2 and n1 == (n2-1) and m1 == (m2-1):
                        H_supper[ind1,ind2] = kappa*np.sqrt(n2*m2)
                    elif i1 == i2 and j1 == j2 and n1 == n2-1 and m1 == m2:
                        H_supper[ind1,ind2] = -kappa/2*np.sqrt(n0*n2)
                    elif i1 == i2 and j1 == j2 and n1 == n2+1 and m1 == m2:
                        H_supper[ind1,ind2] = kappa/2*np.sqrt(n0*n1)
                    elif i1 == i2 and j1 == j2 and n1 == n2 and m1-1 == m2:
                        H_supper[ind1,ind2] = kappa/2*np.sqrt(n0*m1)
                    elif i1 == i2 and j1 == j2 and n1 == n2 and m1+1 == m2:
                        H_supper[ind1,ind2] = -kappa/2*np.sqrt(n0*m2)
                    elif j1 == j2 and n1 == n2 and m1 == m2:
                        H_supper[ind1,ind2] = -1j*rotation[i1,i2]
                    elif i1 == i2 and n1 == n2 and m1 == m2:
                        H_supper[ind1,ind2] = 1j*np.conjugate(rotation[j2,j1]) #?
                if np.mod(ind1,10) == 0:
                    print("\r", end="")
                    print(f"H_supper Generating{ind1+1}/{lenrs}", end="")
                    sys.stdout.flush()
            
            print("\n")
            print("U_supper Generating")
            eigvalue,eigvector = np.linalg.eig(H_supper)
            eigvalue[np.real(eigvalue)>30] = 0
            H_supper = eigvector@np.diag(eigvalue)@np.linalg.inv(eigvector)
            U_supper = expm(H_supper*photons[i].t)
            print("U_supper Appling")
            rho_supper = U_supper@rho_supper
            print(f"step {i}/{sequence_length} done")
            n_p = 0
            atom_number = self.atom_number
            for ind1 in range(len(rho_supper)):
                i1 = int(ind1/nc**2/(atom_number+1))
                j1 = int(np.mod(ind1/nc**2,(atom_number+1)))
                n1 = int(np.mod(ind1/nc,nc))
                m1 = int(np.mod(ind1,nc))
                if n1 == m1 and i1 == j1:
                    n_p+=np.abs(n1*rho_supper[ind1])
            num_phoc = np.append(num_phoc,n_p)
            if plot:
                lenrs = (atom_number+1)**2*nc**2
                mzs = np.arange(atom_number+1)
                nps = np.arange(nc)
                mzs,nps = np.meshgrid(mzs,nps)
                mzs,nps = mzs.ravel(), nps.ravel()
                rhos = np.array(mzs+nps,dtype = float)
                bottom = np.zeros_like(rhos)
                width = 0.8
                depth = 0.8
                for ind1 in range(lenrs):
                    i1 = int(ind1/nc**2/(atom_number+1))
                    j1 = int(np.mod(ind1/nc**2,(atom_number+1)))
                    n1 = int(np.mod(ind1/nc,nc))
                    m1 = int(np.mod(ind1,nc))
                    if i1 == j1 and n1 == m1:
                        rhos[int(i1+n1*(atom_number+1))] = np.abs(rho_supper[ind1])

                fig = plt.figure(figsize=(4, 3))
                ax = fig.add_subplot(111, projection='3d')
                ax.bar3d(mzs,nps,bottom,width,depth,rhos,shade = True)
                ax.set_xlabel("Sz")
                ax.set_ylabel("N_photons")
                ax.set_zlabel("rho")
                ax.set_title("rho diagram")
                ax.set_zlim(0,1)
                plt.savefig(f"fig/gif{i}.jpg")
                plt.close()
                image_list.append(f"fig/gif{i}.jpg")
        if plot:
            gif_name = "fig/gif.gif"
            duration = 0.05
            create_gif(image_list, gif_name,duration)

        return rho_supper,num_phoc
    
    def Lindblad_viewsp(self,nc,sequence,num_frame = 31,plot = False):
        rho = self.rho
        rho_supper = np.zeros((self.atom_number+1)**2*nc**2,dtype = complex)
        lenrs = len(rho_supper)
        for ind1 in range(lenrs):
            i1 = int(ind1/nc**2/(self.atom_number+1))
            j1 = int(np.mod(ind1/nc**2,(self.atom_number+1)))
            n1 = int(np.mod(ind1/nc,nc))
            m1 = int(np.mod(ind1,nc))
            if n1 == 0 and m1 == 0:
                rho_supper[ind1] = rho[i1,j1]
        photons = sequence.photons
        sequence_length = len(sequence.photons)
        num_phoc = np.array([],dtype = float)
        image_list = []
        for i in range(sequence_length):
            rotation = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
            rho0_supper = rho_supper*1.0
            
            no = photons[i].order[0]
            n0 = photons[i].delta[0]/self.omega_s

            if photons[i].typ == "x":
                rotation += self.Omega*self.S_x
            elif photons[i].typ == "z":
                rotation += self.Omega*self.S_z
            elif photons[i].typ == "y":
                rotation += self.Omega*self.S_y
            H_supper = np.zeros([lenrs,lenrs],dtype = complex)
            for ind1 in range(lenrs):
                i1 = int(ind1/nc**2/(self.atom_number+1))
                j1 = int(np.mod(ind1/nc**2,(self.atom_number+1)))
                n1 = int(np.mod(ind1/nc,nc))
                m1 = int(np.mod(ind1,nc))
                for ind2 in range(lenrs):
                    i2 = int(ind2/nc**2/(self.atom_number+1))
                    j2 = int(np.mod(ind2/nc**2,(self.atom_number+1)))
                    n2 = int(np.mod(ind2/nc,nc))
                    m2 = int(np.mod(ind2,nc))
                    if i1 == i2 and j1 == j2 and n1 == n2 and m1 == m2:
                        H_supper[ind1,ind2] = -kappa*(n1+m1)/2-1j*(self.omega_s*((i1-no)*n1-(j2-no)*m2))*(1-1j*Gamma/(2*self.Delta))
                    elif i1 == i2 and j1 == j2 and n1 == (n2-1) and m1 == (m2-1):
                        H_supper[ind1,ind2] = kappa*np.sqrt(n2*m2)
                    elif i1 == i2 and j1 == j2 and n1 == n2-1 and m1 == m2:
                        H_supper[ind1,ind2] = -kappa/2*np.sqrt(n0*n2)
                    elif i1 == i2 and j1 == j2 and n1 == n2+1 and m1 == m2:
                        H_supper[ind1,ind2] = kappa/2*np.sqrt(n0*n1)
                    elif i1 == i2 and j1 == j2 and n1 == n2 and m1-1 == m2:
                        H_supper[ind1,ind2] = kappa/2*np.sqrt(n0*m1)
                    elif i1 == i2 and j1 == j2 and n1 == n2 and m1+1 == m2:
                        H_supper[ind1,ind2] = -kappa/2*np.sqrt(n0*m2)
                    elif j1 == j2 and n1 == n2 and m1 == m2:
                        H_supper[ind1,ind2] = -1j*rotation[i1,i2]
                    elif i1 == i2 and n1 == n2 and m1 == m2:
                        H_supper[ind1,ind2] = 1j*np.conjugate(rotation[j2,j1]) #?
                if np.mod(ind1,10) == 0:
                    print("\r", end="")
                    print(f"H_supper Generating{ind1+1}/{lenrs}", end="")
                    sys.stdout.flush()
            
            print("\n")
            print("U_supper Generating")
            eigvalue,eigvector = np.linalg.eig(H_supper)
            eigvalue[np.real(eigvalue)>30] = 0
            H_supper = eigvector@np.diag(eigvalue)@np.linalg.inv(eigvector)
            if plot:
                atom_number = self.atom_number
                U_supper = expm(H_supper*photons[i].t/(num_frame-1))
                for j in range(num_frame):
                    rho_supper = U_supper@rho_supper
                    lenrs = (atom_number+1)**2*nc**2
                    mzs = np.arange(atom_number+1)
                    nps = np.arange(nc)
                    mzs,nps = np.meshgrid(mzs,nps)
                    mzs,nps = mzs.ravel(), nps.ravel()
                    rhos = np.array(mzs+nps,dtype = float)
                    bottom = np.zeros_like(rhos)
                    width = 0.8
                    depth = 0.8
                    for ind1 in range(lenrs):
                        i1 = int(ind1/nc**2/(atom_number+1))
                        j1 = int(np.mod(ind1/nc**2,(atom_number+1)))
                        n1 = int(np.mod(ind1/nc,nc))
                        m1 = int(np.mod(ind1,nc))
                        if i1 == j1 and n1 == m1:
                            rhos[int(i1+n1*(atom_number+1))] = np.abs(rho_supper[ind1])

                    fig = plt.figure(figsize=(4, 3))
                    ax = fig.add_subplot(111, projection='3d')
                    ax.bar3d(mzs,nps,bottom,width,depth,rhos,shade = True)
                    ax.set_xlabel("Sz")
                    ax.set_ylabel("N_photons")
                    ax.set_zlabel("rho")
                    ax.set_title("rho diagram")
                    ax.set_zlim(0,1)
                    plt.savefig(f"fig/gif{i}_{j}.jpg")
                    plt.close()
                    image_list.append(f"fig/gif{i}_{j}.jpg")
                    n_p = 0
                    for ind1 in range(len(rho_supper)):
                        i1 = int(ind1/nc**2/(atom_number+1))
                        j1 = int(np.mod(ind1/nc**2,(atom_number+1)))
                        n1 = int(np.mod(ind1/nc,nc))
                        m1 = int(np.mod(ind1,nc))
                        if n1 == m1 and i1 == j1:
                            n_p+=np.abs(n1*rho_supper[ind1])
                    num_phoc = np.append(num_phoc,n_p)
            else:
                U_supper = expm(H_supper*photons[i].t)
                print("U_supper Appling")
                rho_supper = U_supper@rho_supper
                print(f"step {i}/{sequence_length} done")
                n_p = 0
                atom_number = self.atom_number
                for ind1 in range(len(rho_supper)):
                    i1 = int(ind1/nc**2/(atom_number+1))
                    j1 = int(np.mod(ind1/nc**2,(atom_number+1)))
                    n1 = int(np.mod(ind1/nc,nc))
                    m1 = int(np.mod(ind1,nc))
                    if n1 == m1 and i1 == j1:
                        n_p+=np.abs(n1*rho_supper[ind1])
                num_phoc = np.append(num_phoc,n_p)
            
        if plot:
            gif_name = "fig/gif.gif"
            duration = 0.05
            create_gif(image_list, gif_name,duration)

        return rho_supper,num_phoc
    
    def Lindblad_Approximate(self,nc,sequence,enable_pumping = True):
        rho = self.rho
        rho_supper = np.zeros((self.atom_number+1)**2*nc**2,dtype = complex)
        lenrs = len(rho_supper)
        for ind1 in range(lenrs):
            i1 = int(ind1/nc**2/(self.atom_number+1))
            j1 = int(np.mod(ind1/nc**2,(self.atom_number+1)))
            n1 = int(np.mod(ind1/nc,nc))
            m1 = int(np.mod(ind1,nc))
            if n1 == 0 and m1 == 0:
                rho_supper[ind1] = rho[i1,j1]
        photons = sequence.photons
        sequence_length = len(sequence.photons)
        for i in range(sequence_length):
            H_AC = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
            rho0_supper = rho_supper*1.0
            
            for j in range(len(photons[i].delta)):
                H_AC += self.H_AC(photons[i].delta[j], photons[i].order[j])
            
            if enable_pumping:
                trace_test = 0
                for ind1 in range(lenrs):
                    i1 = int(ind1/nc**2/(self.atom_number+1))
                    j1 = int(np.mod(ind1/nc**2,(self.atom_number+1)))
                    n1 = int(np.mod(ind1/nc,nc))
                    m1 = int(np.mod(ind1,nc))
                    ki = np.real(H_AC[i1,i1])/self.omega_s
                    kj = np.real(H_AC[j1,j1])/self.omega_s
                    if np.abs(ki-n1)>100:
                        coei = 0
                    elif n1>50:
                        coei = np.sqrt((ki/n1)**n1/np.sqrt(2*np.pi*n1)*np.exp(n1-ki))
                    else:
                        coei = np.sqrt(ki**n1/np.math.factorial(n1)*np.exp(-ki))
                    if np.abs(kj-m1)>100:
                        coej = 0
                    elif m1>50:
                        coej = np.sqrt((kj/m1)**m1/np.sqrt(2*np.pi*m1)*np.exp(m1-kj))
                    else:
                        coej = np.sqrt(kj**m1/np.math.factorial(m1)*np.exp(-kj))
                    rho_supper[ind1] = rho0_supper[i1*nc**2*(atom_number+1)+j1*nc**2]*coei*coej
#                     if i1 == j1 and n1 == m1:
#                         trace_test+=rho_supper[ind1]
                    
            
            if photons[i].typ == "x":
                H_AC += self.Omega*self.S_x
            elif photons[i].typ == "z":
                H_AC += self.Omega*self.S_z
            elif photons[i].typ == "y":
                H_AC += self.Omega*self.S_y
            H_supper = np.zeros([lenrs,lenrs],dtype = complex)
            for ind1 in range(lenrs):
                i1 = int(ind1/nc**2/(self.atom_number+1))
                j1 = int(np.mod(ind1/nc**2,(self.atom_number+1)))
                n1 = int(np.mod(ind1/nc,nc))
                m1 = int(np.mod(ind1,nc))
                for ind2 in range(lenrs):
                    i2 = int(ind2/nc**2/(self.atom_number+1))
                    j2 = int(np.mod(ind2/nc**2,(self.atom_number+1)))
                    n2 = int(np.mod(ind2/nc,nc))
                    m2 = int(np.mod(ind2,nc))
                    if i1 == i2 and j1 == j2 and n1 == n2 and m1 == m2:
                        H_supper[ind1,ind2] = -kappa*(n1+m1)/2-1j*H_AC[i1,i2]+1j*np.conjugate(H_AC[j1,j2])
                    elif i1 == i2 and j1 == j2 and n1 == (n2-1) and m1 == (m2-1):
                        H_supper[ind1,ind2] = kappa*np.sqrt(n2)*np.sqrt(m2)
                    elif j1 == j2 and n1 == n2 and m1 == m2:
                        H_supper[ind1,ind2] = -1j*H_AC[i1,i2]
                    elif i1 == i2 and n1 == n2 and m1 == m2:
                        H_supper[ind1,ind2] = 1j*np.conjugate(H_AC[j2,j1]) #?
                if np.mod(ind1,10) == 0:
                    print("\r", end="")
                    print(f"H_supper Generating{ind1+1}/{lenrs}", end="")
                    sys.stdout.flush()
            
            print("\n")
            print("U_supper Generating")
            U_supper = expm(H_supper*photons[i].t)
            print("U_supper Appling")
            rho_supper = U_supper@rho_supper
        return rho_supper
                    
            
    
            
    def evaluation(self, mode = "SSS"):
        if mode == "GHZ":
            density_mat = self.psi@np.transpose(np.conj(self.psi))
            fidelity = 1/2 * np.abs(density_mat[0,0] + density_mat[self.atom_number, self.atom_number] + np.abs(density_mat[0, self.atom_number] + density_mat[self.atom_number, 0]) )
            return 1-fidelity,0
        elif mode == "SSS": 
            # This is ξ_R^2, which is sensitive to the unitarity for the most.
            rho = self.rho
            loss_atom_num = (1-np.real(np.trace(rho)))*self.atom_number
            print(loss_atom_num)
            Jzmean = np.real(np.trace(self.S_z@rho))
            Jxmean = np.real(np.trace(self.S_x@rho))
            Jymean = np.real(np.trace(self.S_y@rho))
            Jabs = np.real(np.sqrt(Jzmean**2 + Jxmean**2 + Jymean**2))
            Polar_Angle = np.arccos(Jzmean/Jabs)
            if Polar_Angle == 0:
                Azimuthal_Angle = 0
            else:
                temp = Jxmean/(Jabs*np.sin(Polar_Angle))
                if temp>1:temp-=1e-6
                if temp<-1:temp+=1e-6
                Azimuthal_Angle = np.arccos(temp)
                if Jymean<=0: Azimuthal_Angle = 2*np.pi - Azimuthal_Angle
            n_1 = np.array([-np.sin(Azimuthal_Angle), np.cos(Azimuthal_Angle), 0])
            n_2 = np.array([np.cos(Polar_Angle)*np.cos(Azimuthal_Angle), np.cos(Polar_Angle)*np.sin(Azimuthal_Angle), -np.sin(Polar_Angle)])
            J_n1 = self.S_x*n_1[0] + self.S_y*n_1[1] + self.S_z*n_1[2]
            J_n2 = self.S_x*n_2[0] + self.S_y*n_2[1] + self.S_z*n_2[2]
            C = np.real(np.trace(rho@(J_n1@J_n1+J_n2@J_n2)))
            A = np.real(np.trace(rho@(J_n1@J_n1 - J_n2@J_n2)))
            B = np.real(np.trace(rho@J_n1@J_n2+rho@J_n2@J_n1))
            min_variance = 1/2*(C-np.sqrt(A**2+B**2))
            variance = min_variance +1/4*np.sqrt(loss_atom_num)
            Squeezing_Parameter = self.atom_number/(Jabs**2) * variance
            
            return Squeezing_Parameter, Jabs, Polar_Angle, Azimuthal_Angle
        elif mode == "SSSN": 
            # This is ξ_R^2, which is sensitive to the unitarity for the most.
            rho = self.rho
            loss_atom_num = (1-np.real(np.trace(rho)))*self.atom_number
            Jzmean = np.real(np.trace(self.S_z@rho))
            Jxmean = np.real(np.trace(self.S_x@rho))
            Jymean = np.real(np.trace(self.S_y@rho))
            Jabs = np.real(np.sqrt(Jzmean**2 + Jxmean**2 + Jymean**2))
            Polar_Angle = np.arccos(Jzmean/Jabs)
            if Polar_Angle == 0:
                Azimuthal_Angle = 0
            else:
                temp = Jxmean/(Jabs*np.sin(Polar_Angle))
                if temp>1:temp-=1e-6
                if temp<-1:temp+=1e-6
                Azimuthal_Angle = np.arccos(temp)
                if Jymean<=0: Azimuthal_Angle = 2*np.pi - Azimuthal_Angle
            n_1 = np.array([-np.sin(Azimuthal_Angle), np.cos(Azimuthal_Angle), 0])
            n_2 = np.array([np.cos(Polar_Angle)*np.cos(Azimuthal_Angle), np.cos(Polar_Angle)*np.sin(Azimuthal_Angle), -np.sin(Polar_Angle)])
            J_n1 = self.S_x*n_1[0] + self.S_y*n_1[1] + self.S_z*n_1[2]
            J_n2 = self.S_x*n_2[0] + self.S_y*n_2[1] + self.S_z*n_2[2]
            C = np.real(np.trace(rho@(J_n1@J_n1+J_n2@J_n2)))
            A = np.real(np.trace(rho@(J_n1@J_n1 - J_n2@J_n2)))
            B = np.real(np.trace(rho@J_n1@J_n2+rho@J_n2@J_n1))
            print(C)
            print(A)
            print(B)
            min_variance = 1/2*(C-np.sqrt(A**2+B**2))
            variance = min_variance
            Squeezing_Parameter = np.real(np.trace(rho))*self.atom_number/(Jabs**2) * variance
            
            return Squeezing_Parameter, Jabs, Polar_Angle, Azimuthal_Angle
        
        else:
            p = re.compile('(N\d*)*')
            m = p.match(mode)
            if not m==None:
                rho = self.rho
                nums = np.array([],dtype = int)
                pnum = re.compile("\d\d*")
                endmark = 0
                counter = 0
                while endmark<(len(mode)-1):
                    mnum = pnum.search(mode,endmark)
                    num = mode[mnum.span()[0]:mnum.span()[1]]
                    nums = np.append(nums,int(num))
                    endmark = mnum.span()[1]
                    counter += 1
                fidelity = 0.0
                for i in range(counter):
                    for j in range(counter):
                        fidelity += np.real(rho[nums[i],nums[j]])/counter
                return 1-fidelity,0
            
            
    def calc_SP_from_scratch(self, unitarity=0.95):
        rho = self.psi @ np.transpose(np.conj(self.psi))
        Jzmean = np.real(np.trace(self.S_z @ rho))
        Jxmean = np.real(np.trace(self.S_x @ rho))
        Jymean = np.real(np.trace(self.S_y @ rho))
        # print([Jzmean,Jxmean,Jymean])
        Jabs = np.real(np.sqrt(Jzmean**2 + Jxmean**2 + Jymean**2))
        # print("Jabs:",Jabs)
        if Jabs < self.atom_number/2*unitarity :
            return [1000,Jabs]
        else:
            Polar_Angle = np.arccos(Jzmean/Jabs)
            if Polar_Angle == 0:
                Azimuthal_Angle = 0
            else:
                temp = Jxmean/(Jabs*np.sin(Polar_Angle))
                if temp>1:temp-=1e-6
                if temp<-1:temp+=1e-6
                Azimuthal_Angle = np.arccos(temp)
                if Jymean<=0: Azimuthal_Angle = 2*np.pi - Azimuthal_Angle
            # print("Jz,Jx,Jy,Jabs:",Jzmean,Jxmean,Jymean,Jabs )
            n1 = np.array([-np.sin(Azimuthal_Angle), np.cos(Azimuthal_Angle), 0])
            n2 = np.array([np.cos(Polar_Angle) * np.cos(Azimuthal_Angle), np.cos(Polar_Angle) * np.sin(Azimuthal_Angle), -np.sin(Polar_Angle)])
            sin_phi = np.array([np.sin(phi) for phi in np.arange(0, np.pi, 0.001)])
            cos_phi = np.array([np.cos(phi) for phi in np.arange(0, np.pi, 0.001)])

            # Calculating variances for different directions
            variances = []
            for sinp, cosp in zip(sin_phi, cos_phi):
                direction = sinp * n1 + cosp * n2
                J_n = self.S_x * direction[0] + self.S_y * direction[1] + self.S_z * direction[2]
                var = np.real(np.trace(rho @ (J_n @ J_n))) - np.real(np.trace(rho @ J_n)) ** 2
                variances.append(var)

            # Finding the minimum variance
            min_variance = min(variances)
            print("Min_Variance:", min_variance)

            # Calculating squeezing parameters
            xi_S_squared = 4 * min_variance / self.atom_number/np.real(np.trace(rho))
            xi_R_squared = (np.real(np.trace(rho))**2*self.atom_number ** 2 / (4 * Jabs ** 2)) * xi_S_squared

            return [xi_R_squared, xi_S_squared, Jabs]


    # Cavity amplitude transmission function t
    def t(self, ksi, n_up):   # ksi is the detuning between light and cavity mode ω-ω_c
        FSR = kappa*self.cooperativity*k0w0**2/24*np.pi*2
        fpg1 = 0
        fpg2 = 0
        for i in range(11):
            fpg1 += (n_up*self.cooperativity) / (1+4*(self.Delta+ksi-FSR*(i-5))**2/Gamma**2)+2*1j*n_up*self.cooperativity*(self.Delta+ksi-FSR*(i-5))/Gamma / (1+4*(self.Delta+ksi-FSR*(i-5))**2/Gamma**2)
            fpg2 += ((self.atom_number-n_up)*self.cooperativity) / (1+4*(-Omega_a+self.Delta+ksi-FSR*(i-5))**2/Gamma**2)+2*1j*(self.atom_number-n_up)*self.cooperativity*(-Omega_a+self.Delta+ksi-FSR*(i-5))/Gamma / (1+4*(-Omega_a+self.Delta+ksi-FSR*(i-5))**2/Gamma**2)
        return 1 / (1+fpg1+fpg2-2*1j*ksi/kappa)
    
    def show_occupation(self):
        plt.figure()
        plt.scatter(self.state_index-self.atom_number/2, np.abs(np.diag(self.rho)))     # Occupation is the probability distribution, for sure
        plt.show()

    def system_reset(self):
        self.psi = deepcopy(self.psi0)

    def husimi_q(self, theta, phi):
        q_func = np.zeros((len(theta), len(phi)), dtype=np.complex128)
        for j, t in enumerate(theta):
            for k, p in enumerate(phi):
                alpha = np.cos(t / 2) * np.exp(1j * p / 2)
                beta = np.sin(t / 2) * np.exp(-1j * p / 2)
                coherent_state = np.array([alpha ** m * beta ** (self.atom_number - m) * np.sqrt(factorial(self.atom_number) / (factorial(m) * factorial(self.atom_number - m))) for m in range(self.atom_number + 1)]).reshape(-1, 1)
                q_func[k, j] = np.abs(np.conj(coherent_state).T @ self.rho@coherent_state)
        return np.abs(q_func)

    def visualize(self):
        theta = np.linspace(0, np.pi, 100)
        phi = np.linspace(0, np.pi*4/2, 100)
        theta_grid, phi_grid = np.meshgrid(theta, phi)
        q_func = self.husimi_q(theta, phi)
        x = np.sin(theta_grid) * np.cos(phi_grid)
        y = np.sin(theta_grid) * np.sin(phi_grid)
        z = -np.cos(theta_grid)
        # Normalize the q_func values for the colormap
        norm = Normalize(vmin=np.min(q_func), vmax=np.max(q_func))
        

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=plt.cm.jet(norm(q_func)), alpha=0.8, linewidth=0, antialiased=True,shade = False)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title('Husimi-Q Representation')
#         ax.set_aspect('equal')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])
        plt.axis('off')
        plt.show()
        
    def rgb(self,colors,corders,norm):
        count = np.zeros(norm.shape,dtype = int)
        for i in range(len(corders)-1):
            count[norm>corders[i+1]]+=1
        recolors = np.zeros(norm.shape+(3,))
        recolors[...,0] = colors[count,0]*(corders[count+1]-norm)/(corders[count+1]-corders[count]) +colors[count+1,0]*(norm-corders[count])/(corders[count+1]-corders[count])
        recolors[...,1] = colors[count,1]*(corders[count+1]-norm)/(corders[count+1]-corders[count]) +colors[count+1,1]*(norm-corders[count])/(corders[count+1]-corders[count])
        recolors[...,2] = colors[count,2]*(corders[count+1]-norm)/(corders[count+1]-corders[count]) +colors[count+1,2]*(norm-corders[count])/(corders[count+1]-corders[count])
        return recolors
    
    def visualize_withwall(self,diag):
        theta = np.linspace(0, np.pi, 100)
        phi = np.linspace(0, np.pi*4/2, 100)
        theta_grid, phi_grid = np.meshgrid(theta, phi)
        q_func = self.husimi_q(theta, phi)
        x = np.sin(theta_grid) * np.cos(phi_grid)
        y = np.sin(theta_grid) * np.sin(phi_grid)
        z = -np.cos(theta_grid)
        # Normalize the q_func values for the colormap
        norm = Normalize(vmin=np.min(q_func), vmax=np.max(q_func))
        q_norm = norm(q_func)
#         for i in range(200):
#             print(q_norm[i])
#         clsphi = np.array([[245,245,245],[237,90,179],[22,64,214],[0,27,121]])/256
        clsphi = np.array([[240,240,240],[232,218,184],[240,200,0],[230,100,0],[250,0,0]])/256
        corderphi = np.array([0,0.05,0.2,0.5,1])
        colors_phi = np.zeros(x.shape + (3,))
        colors_phi = self.rgb(clsphi,corderphi,q_norm)
#         colors_phi[..., 0] = np.cos(q_norm*np.pi)**2*0.9
#         colors_phi[..., 1] = (1-np.sin(q_norm*np.pi*3/2)**2)*0.9
#         colors_phi[..., 2] = (1-np.sin(q_norm*np.pi*2/2)**2)*0.9
        
        diagabs= np.abs(diag)
        wall = np.zeros(x.shape)
        for i in range(len(theta)):
            for j in range(len(phi)):
                rightwhere = (1-np.cos(theta[i]))*self.atom_number/2*0.999999
                rightint = int(rightwhere)
                wall[j,i] = diagabs[rightint]*(rightint+1-rightwhere) + diagabs[rightint+1]*(rightwhere-rightint)
        norm_wall = Normalize(vmin=np.min(wall), vmax=np.max(wall))
        q_wall = norm_wall(wall)
        colors_wall = np.zeros(x.shape + (3,))
        colors_wall[..., 0] = 1-q_wall*0.4
        colors_wall[..., 1] = 1-q_wall*0.4
        colors_wall[..., 2] = 1
        
        colors = np.zeros(x.shape + (3,))
        colors[..., 0] = np.minimum(colors_wall[...,0],colors_phi[...,0])
        colors[..., 1] = np.minimum(colors_wall[...,1],colors_phi[...,1])
        colors[..., 2] = np.minimum(colors_wall[...,2],colors_phi[...,2])

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        surf = ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=colors, alpha=0.7, linewidth=0, antialiased=True,shade=False)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title('Husimi-Q Representation')
        plt.axis('off')
        plt.show()

# gif generation
def create_gif(image_list, gif_name, duration=0.05):
    frames = []

    for i in range(int(1/duration)):
        frames.append(imageio.imread(image_list[0]))

    for image_name in image_list:
        frames.append(imageio.imread(image_name))

    for i in range(int(1/duration)):
        frames.append(imageio.imread(image_list[-1]))
    imageio.mimsave(gif_name, frames, 'GIF', duration=duration)
    return None


# 10 atoms 3 photons max
delta0 =  np.array([0.])
order0 =  np.array([28.96868897])
t0 =  0
typ0 = "x"
delta1 =  np.array([-367524.55058617])*30
order1 =  np.array([2])
t1 =  7.336659061816678e-05*120
typ1 = "x"
Delta =  -4100000000.0
atom_number = 10
target = "N1N1"
nc = 4

ini_sequence = sequence(np.array([photons(typ1,order1,delta1,t1/2),photons(typ1,order1,delta1,t1/2)]))

test_photons = np.array([],dtype = object)
num_frames = 31
for i in range(num_frames):
    test_photons = np.append(test_photons,photons(typ1,order1,delta1,t1/(num_frames-1)))

test_sequence = sequence(test_photons)
system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 30000, Delta = Delta, Omega = 0.2*10**4*2*np.pi/30)
rho_supper,num_photons = system.Lindblad_viewsp(nc,ini_sequence,num_frame = 16, plot = True)

# 20 atoms 3 photons max
delta0 =  np.array([0.])
order0 =  np.array([28.96868897])
t0 =  0
typ0 = "x"
delta1 =  np.array([-367524.55058617])*30
order1 =  np.array([2])
t1 =  7.336659061816678e-05*120
typ1 = "x"
Delta =  -4100000000.0
atom_number = 20
target = "N1N1"
nc = 4

ini_sequence = sequence(np.array([photons(typ1,order1,delta1,t1/2),photons(typ1,order1,delta1,t1/2)]))

test_photons = np.array([],dtype = object)
num_frames = 31
for i in range(num_frames):
    test_photons = np.append(test_photons,photons(typ1,order1,delta1,t1/(num_frames-1)))

test_sequence = sequence(test_photons)
system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 30000, Delta = Delta, Omega = 0.2*10**4*2*np.pi/30)
rho_supper,num_photons = system.Lindblad_viewsp(nc,ini_sequence,num_frame = 16, plot = True)

# 10 atoms GHZ demo
delta0 =  np.array([0.])
order0 =  np.array([3.00191979])
coe = 100
t0 =  3.0566478623073844e-07*2*coe
typ0 = "x"
delta1 =  np.array([-5676969.374851/2/coe*75])
order1 =  np.array([4.96743527])
t1 =  0.0000021214936243843288*2*coe
typ1 = "x"
Delta =  -4636669274.663687
atom_number = 10
ini_sequence = sequence(np.array([photons(typ0,order0,delta0,t0),photons(typ1,order1,delta1,t1)]))
target = "N0N10"

test_photons = np.array([photons(typ0,order0,delta0,t0)],dtype = object)
num_frames = 21
for i in range(num_frames):
    test_photons = np.append(test_photons,photons(typ1,order1,delta1,t1/(num_frames-1)))

test_sequence = sequence(test_photons)

system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 10000, Delta = Delta, Omega = 0.1*10**6*2*np.pi/coe)
nc = 3
rho_supper,num_photons = system.Lindblad(nc,test_sequence,plot = True)

Using matplotlib backend: <object object at 0x00000201DC8216E0>
H_supper Generating341/1936

KeyboardInterrupt: 